In [77]:
import sys
sys.path.append("c:/Users/oliver/Projects/CLT/")

import math

from clt import Laminate
from material import TransverseIsotropicPlyMaterial
from plainstrength import PlainStrengthAnalysisA

In [65]:
matmg = TransverseIsotropicPlyMaterial(t=0.184, 
                                       E11=154000, E22=154000, G12=4200, nu12=0.35, 
                                       a11t=0.15e-6, a22t=28.7e-6, 
                                       F11t=2610, F11c=-1450, F22t=50, F22c=-120, F12s=105)
puck = PlainStrengthAnalysisA(lam, 'Puck')

lam = Laminate([(x, matmg) for x in (45, -45, 0, 90, 90, 0, -45, 45)])
load = [500, 100, -10, 0, 0, 0]

In [66]:
lam.thickness()

1.4719999999999998

In [67]:
res = puck.all_rf(load, 0)

In [68]:
[r for r in res if '_MF_' not in r.fmode and r.r is not None]

[StrengthResult(lid=0, angle=45.0, eta=0.5, r=13.654250941402164, fmode='Puck_FF_T'),
 StrengthResult(lid=1, angle=-45.0, eta=0.5, r=12.057687013472991, fmode='Puck_FF_T'),
 StrengthResult(lid=2, angle=0.0, eta=0.5, r=5.712335115344682, fmode='Puck_FF_T'),
 StrengthResult(lid=3, angle=90.0, eta=0.5, r=29.41337195925647, fmode='Puck_FF_C'),
 StrengthResult(lid=4, angle=90.0, eta=0.5, r=29.41335447743833, fmode='Puck_FF_C'),
 StrengthResult(lid=5, angle=0.0, eta=0.5, r=5.712333953373845, fmode='Puck_FF_T'),
 StrengthResult(lid=6, angle=-45.0, eta=0.5, r=12.057686748454065, fmode='Puck_FF_T'),
 StrengthResult(lid=7, angle=45.0, eta=0.5, r=13.654250576856676, fmode='Puck_FF_T')]

## Orthotropic Plate buckling, compression

#### HSB 45111-08

- prerequisites: 
  - $B$ matrix is zero
  - special orthotropic: $D_{16} = D_{26} = 0$
  - plate is thin: $t/a \lt 0.5$, $t/b \lt 0.5$
- figure 1: all edges simply supported
- length of edge  parallel to load $a$
- length of edge perpendicular to load $b$
- effective aspect ratio $\bar \alpha = \frac{a}{b} \cdot \left( \frac{D_{22}}{D_{11}} \right)^{1/4}$
- orthotropy parameter $\beta = \frac{D_{12} + 2 D_{66}}{\sqrt{D_{11} D_{22}}}$
- Buckling coefficient, lower bound: $k_{x, LB} = 2 (1 + \beta)$  for $\bar \alpha \ge 1$
- Buckling load $n_{cr} = k_x \left( \frac{\pi}{b} \right)^2 \sqrt{D_{11} D_{22}}$ for $\bar \alpha \ge 1$

#### simplified analytical formulas, also for combined loads

- see Kassapoglou!
- all edges simply supported
- compression
- shear
- combined shear/compression

#### Other simplifications

There is a formula for combined buckling -> paper TU delft 

In [72]:
a = 600
b = 200

In [75]:
D = lam.D()
D

array([[ 3.4357938e+04,  2.8614363e+04, -2.6872715e-14],
       [ 2.8614363e+04,  3.4357938e+04,  2.6872715e-14],
       [-2.6872715e-14,  2.6872715e-14,  1.3404540e+04]], dtype=float32)

In [76]:
alphabar = (a/b) * (D[1,1]/D[0,0])**0.25
alphabar

3.0

In [78]:
beta = (D[0,1] + 2*D[2,2])/(math.sqrt(D[0,0]*D[1,1]))
beta

1.6131190658960939

In [79]:
kx = 2*(1 + beta)
kx

5.226238131792188

In [80]:
ncr = kx * (math.pi/b)**2 * math.sqrt(D[0,0]*D[1,1])
ncr

44.305335373192655

In [82]:
sigcr = ncr / lam.thickness()
sigcr

30.09873326983197

## Stacking sequence constraints: 
- symmetric laminate, 
- balanced laminate
- 0, 90, +/-
- min 10% in any direction
- +/- or +/90/- on outer plies
- contiguous plies: max 4 (0.125) or 2 (0.18, 0.25)
- place +/- plies closely together
- angle between adjacentt plies should be less than 45° (??? almost always violated)

## Task 1

- plate under compressive loads, edge length and loads given
- plate is dominated by compressive loads (e.g. lower shell)
- more than one load case
- find stacking sequence that minimises mass
- plain strength and compressive buckling

  
## Task 2

- Other tasks could be to find a stacking sequence that closely approximates a given ABD matrix
- objective function: ||ABD_1 - ABD_2|| / ||ABD_2|| 
  (after non-dimensionalisation; A, B, D matrices have different dimensions)
- which norm?